# Project GDP
## GitHub Repo: https://github.com/JackBeerman/GDP

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import io
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

In [2]:
### Goal: Have a dashboard that allows us to select a country, country it trades with, view imports and exports, tarrifs
### Goal: predictive analysis of tarrifs and GDP..? too much

## Data

* The World Bank https://data.worldbank.org/
* World Integrated Trade Solution https://wits.worldbank.org/
* World Trade Organization https://www.wto.org/english/res_e/statis_e/statis_e.htm
* Our World in Data https://ourworldindata.org/trade-and-globalization
* International Monetary Fund https://data.imf.org/?sk=9d6028d4-f14a-464c-a2f2-59b2cd424b85
* Human Development Reports https://hdr.undp.org/

## Preprocess World Bank

In [3]:
noncountries = ["Arab World", "Central Europe and the Baltics",
"Caribbean small states",
"East Asia & Pacific (excluding high income)",
"Early-demographic dividend","East Asia & Pacific",
"Europe & Central Asia (excluding high income)",
"Europe & Central Asia", "Euro area",
"European Union","Fragile and conflict affected situations",
"High income",
"Heavily indebted poor countries (HIPC)","IBRD only",
"IDA & IBRD total",
"IDA total","IDA blend","IDA only",
"Latin America & Caribbean (excluding high income)",
"Latin America & Caribbean",
"Least developed countries: UN classification",
"Low income","Lower middle income","Low & middle income",
"Late-demographic dividend","Middle East & North Africa",
"Middle income",
"Middle East & North Africa (excluding high income)",
"North America","OECD members",
"Other small states","Pre-demographic dividend",
"Pacific island small states",
"Post-demographic dividend",
"Sub-Saharan Africa (excluding high income)",
"Sub-Saharan Africa",
"Small states","East Asia & Pacific (IDA & IBRD)",
"Europe & Central Asia (IDA & IBRD)",
"Latin America & Caribbean (IDA & IBRD)",
"Middle East & North Africa (IDA & IBRD)","South Asia",
"South Asia (IDA & IBRD)",
"Sub-Saharan Africa (IDA & IBRD)",
"Upper middle income", "World"]

In [4]:
exports = pd.read_csv("exports/goods_exported.csv", skiprows=4)
#exports.head()

In [5]:
#print(exports.columns)

In [6]:
exports = exports.drop(columns=['Indicator Name', 'Unnamed: 67'])
#print(exports.columns)

In [7]:
exports = exports.rename(columns={"Country Code": "country_code", "Country Name": "country_name_exp", "Indicator Code": 'feature'})
#exports

In [8]:
exports = exports.query('country_name_exp != @noncountries')
#exports

In [9]:
imports = pd.read_csv("imports/import_goods_services.csv", skiprows=4)
#imports.head()

In [10]:
#print(imports.columns)

In [11]:
imports = imports.drop(columns=['Indicator Name', 'Unnamed: 67'])
#print(imports.columns)

In [12]:
imports = imports.rename(columns={"Country Code": "country_code", "Country Name": "country_name_imp", "Indicator Code": 'feature'})
#imports

In [13]:
imports = imports.query('country_name_imp != @noncountries')
#imports

#### GDP

In [14]:
gdp = pd.read_csv("gdp/GDP.csv", skiprows=4)
#gdp.head()

In [15]:
#print(gdp.columns)

In [16]:
gdp = gdp.drop(columns=['Indicator Name', 'Unnamed: 67'])
#print(gdp.columns)

In [17]:
gdp = gdp.rename(columns={"Country Code": "country_code", "Country Name": "country_name_gdp", "Indicator Code": 'feature'})
#gdp

In [18]:
gdp = gdp.query('country_name_gdp != @noncountries')
#gdp

##### Tariffs

In [19]:
tariffs = pd.read_csv("tariffs/tarrifs.csv", skiprows=4)
#tariffs.head()

In [20]:
#print(tariffs.columns)

In [21]:
tariffs = tariffs.drop(columns=['Indicator Name', 'Unnamed: 67'])
#print(tariffs.columns)

In [22]:
tariffs = tariffs.rename(columns={"Country Code": "country_code", "Country Name": "country_name_tar", "Indicator Code": 'feature'})
#tariffs

In [23]:
tariffs = tariffs.query('country_name_tar != @noncountries')
#tariffs

#### NEED TO CHANGE Features

In [ ]:
replace_map = {
"AG.LND.AGRI.ZS": "agricultural_land",
"AG.LND.FRST.ZS": "forest_area",
"AG.PRD.FOOD.XD": "food_production_index",
"CC.EST": "control_of_corruption",
"EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
"EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
"EG.ELC.ACCS.ZS": "access_to_electricity",
"EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
"EG.ELC.RNEW.ZS": "renewable_electricity_output",
"EG.FEC.RNEW.ZS": "renewable_energy_consumption",
"EG.IMP.CONS.ZS": "energy_imports",
"EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
"EG.USE.PCAP.KG.OE": "energy_use",
"EN.ATM.CO2E.PC": "co2_emissions",
"EN.ATM.METH.PC": "methane_emissions",
"EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
"EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
"EN.CLC.CDDY.XD": "cooling_degree_days",
"EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
"EN.CLC.HEAT.XD": "heat_index_35",
"EN.CLC.MDAT.ZS": "droughts",
"EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
"EN.CLC.SPEI.XD": "mean_drought_index",
"EN.MAM.THRD.NO":"mammal_species",
"EN.POP.DNST":"population_density",
"ER.H2O.FWTL.ZS":"annual_freshwater_withdrawals",
"ER.PTD.TOTL.ZS":"terrestrial_and_marine_protected_areas",
"GB.XPD.RSDV.GD.ZS":"research_and_development_expenditure",
"GE.EST":"government_effectiveness",
"IC.BUS.EASE.XQ":"ease_of_doing_business_rank",
"IC.LGL.CRED.XQ":"strength_of_legal_rights_index",
"IP.JRN.ARTC.SC":"scientific_and_technical_journal_articles",
"IP.PAT.RESD":"patent_applications",
"IT.NET.USER.ZS":"individuals_using_the_internet",
"NV.AGR.TOTL.ZS":"agriculture",
"NY.ADJ.DFOR.GN.ZS":"net_forest_depletion",
"NY.ADJ.DRES.GN.ZS":"natural_resources_depletion",
"NY.GDP.MKTP.KD.ZG":"gdp_growth",
"PV.EST":"political_stability_and_absence_of_violence",
"RL.EST":"rule_of_law",
"RQ.EST":"regulatory_quality",
"SE.ADT.LITR.ZS":"literacy_rate",
"SE.ENR.PRSC.FM.ZS":"gross_school_enrollment",
"SE.PRM.ENRR":"primary_school_enrollment",
"SE.XPD.TOTL.GB.ZS":"government_expenditure_on_education",
"SG.GEN.PARL.ZS":"proportion_of_seats_held_by_women_in_national_parliament",
"SH.DTH.COMM.ZS":"cause_of_death",
"SH.DYN.MORT":"mortality_rate",
"SH.H2O.SMDW.ZS":"people_using_safely_managed_drinking_water_services",
"SH.MED.BEDS.ZS":"hospital_beds",
"SH.STA.OWAD.ZS":"prevalence_of_overweight",
"SH.STA.SMSS.ZS":"people_using_safely_managed_sanitation_services",
"SI.DST.FRST.20":"income_share_held_by_lowest_20pct",
"SI.POV.GINI":"gini_index",
"SI.POV.NAHC":"poverty_headcount_ratio_at_national_poverty_lines",
"SI.SPR.PCAP.ZG":"annualized_average_growth_rate_in_per_capita_real_survey",
"SL.TLF.0714.ZS":"children_in_employment",
"SL.TLF.ACTI.ZS":"labor_force_participation_rate",
"SL.TLF.CACT.FM.ZS":"ratio_of_female_to_male_labor_force_participation_rate",
"SL.UEM.TOTL.ZS":"unemployment",
"SM.POP.NETM":"net_migration",
"SN.ITK.DEFC.ZS":"prevalence_of_undernourishment",
"SP.DYN.LE00.IN":"life_expectancy_at_birth",
"SP.DYN.TFRT.IN":"fertility_rate",
"SP.POP.65UP.TO.ZS":"population_ages_65_and_above",
"SP.UWT.TFRT":"unmet_need_for_contraception",
"VA.EST":"voice_and_accountability",
"EN.CLC.CSTP.ZS":"coastal_protection",
"SD.ESR.PERF.XQ":"economic_and_social_rights_performance_score",
"EN.CLC.HDDY.XD":"heating_degree_days",
"EN.LND.LTMP.DC":"land_surface_temperature",
"ER.H2O.FWST.ZS":"freshwater_withdrawal",
"EN.H2O.BDYS.ZS":"water_quality",
"AG.LND.FRLS.HA":"tree_cover_loss"
}


In [ ]:
#wb = wb.replace(replace_map)
#wb

# Melt and conform to one data frame

In [24]:
exports = pd.melt(exports, id_vars=['country_name_exp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#exports

In [25]:
exports = exports.pivot_table(index=['country_name_exp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
exports.columns.name = None
 
 
exports = exports.reset_index(drop=True)

In [26]:
exports['year'] = pd.to_numeric(exports['year'])

In [27]:
imports = pd.melt(imports, id_vars=['country_name_imp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#imports

In [28]:
imports = imports.pivot_table(index=['country_name_imp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
imports.columns.name = None
 
 
imports = imports.reset_index(drop=True)

In [29]:
imports['year'] = pd.to_numeric(imports['year'])

# Merge 1

In [30]:
merge1 = pd.merge(exports, imports, on = ['country_code', 'year'],
                  how = 'inner')

In [31]:
gdp = pd.melt(gdp, id_vars=['country_name_gdp', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#gdp

In [32]:
gdp = gdp.pivot_table(index=['country_name_gdp','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
gdp.columns.name = None
 
 
gdp = gdp.reset_index(drop=True)

In [33]:
gdp['year'] = pd.to_numeric(gdp['year'])

In [34]:
tariffs = pd.melt(tariffs, id_vars=['country_name_tar', 'country_code'
                         ,'feature'], var_name='year', value_name='value')
#tariffs

In [35]:
tariffs = tariffs.pivot_table(index=['country_name_tar','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
tariffs.columns.name = None
 
 
tariffs = tariffs.reset_index(drop=True)

In [36]:
tariffs['year'] = pd.to_numeric(tariffs['year'])

# Merge 2

In [37]:
merge2 = pd.merge(gdp, tariffs, on = ['country_code', 'year'],
                  how = 'inner')

# Merge 3

In [38]:
merge = pd.merge(merge1, merge2, on = ['country_code', 'year'],
                  how = 'inner')

In [39]:
merge.columns

Index(['country_name_exp', 'country_code', 'year', 'BX.GSR.MRCH.CD',
       'country_name_imp', 'BM.GSR.TOTL.CD', 'country_name_gdp',
       'NY.GDP.MKTP.CD', 'country_name_tar', 'TM.TAX.MRCH.WM.FN.ZS'],
      dtype='object')

In [40]:
merge

,country_name_exp,country_code,year,BX.GSR.MRCH.CD,country_name_imp,BM.GSR.TOTL.CD,country_name_gdp,NY.GDP.MKTP.CD,country_name_tar,TM.TAX.MRCH.WM.FN.ZS
0,Afghanistan,AFG,2008,5.632992e+08,Afghanistan,3.873397e+09,Afghanistan,1.024977e+10,Afghanistan,5.00
1,Afghanistan,AFG,2012,4.755310e+08,Afghanistan,9.891159e+09,Afghanistan,2.020357e+10,Afghanistan,5.50
2,Afghanistan,AFG,2013,5.057926e+08,Afghanistan,1.018433e+10,Afghanistan,2.056449e+10,Afghanistan,5.50
3,Afghanistan,AFG,2018,8.752427e+08,Afghanistan,8.086339e+09,Afghanistan,1.841886e+10,Afghanistan,5.72
4,Albania,ALB,1997,9.090000e+07,Albania,6.976125e+08,Albania,2.258514e+09,Albania,14.41
...,...,...,...,...,...,...,...,...,...,...
3461,Zimbabwe,ZWE,2010,3.245066e+09,Zimbabwe,6.593559e+09,Zimbabwe,1.204166e+10,Zimbabwe,16.60
3462,Zimbabwe,ZWE,2011,4.527563e+09,Zimbabwe,9.528102e+09,Zimbabwe,1.410192e+10,Zimbabwe,13.49
3463,Zimbabwe,ZWE,2012,3.963773e+09,Zimbabwe,8.683377e+09,Zimbabwe,1.711485e+10,Zimbabwe,14.76
3464,Zimbabwe,ZWE,2015,3.577478e+09,Zimbabwe,7.584347e+09,Zimbabwe,1.996312e+10,Zimbabwe,12.86


## Preprocess Human Development Reports

need to add openpyxl to requirements

In [41]:
hdi = pd.read_excel("HDR21-22_Statistical_Annex_HDI_Table.xlsx")

In [42]:
hdi.head()

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Previous HDI rank
0,1,Switzerland,0.962,83.9872,16.500299,13.85966,66933.00454,5,3
1,2,Norway,0.961,83.2339,18.185200,13.00363,64660.10622,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.76717,55782.04981,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.22621,62606.84540,6,4
4,5,Australia,0.951,84.5265,21.054590,12.72682,49238.43335,18,5


In [43]:
hdi.dtypes

HDI rank                                    int64
Country                                    object
Human Development Index (HDI)             float64
Life expectancy at birth                  float64
Expected years of schooling               float64
Mean years of schooling                   float64
Gross national income (GNI) per capita    float64
GNI per capita rank minus HDI rank          int64
Previous HDI rank                           int64
dtype: object

In [44]:
hdi = hdi.rename(columns={"HDI rank": "rank","Country": "country_name_hdi", "Human Development Index (HDI)": "hdi", "Life expectancy at birth": 'life_exp', "Expected years of schooling": 'exp_years_of_school', "Mean years of schooling": 'mean_years_of_school', "Gross national income (GNI) per capita": 'gni', "GNI per capita rank minus HDI rank": 'gni_minus_hdi', "Previous HDI rank": 'old_hdi' })
hdi

,rank,country_name_hdi,Human Development Index (HDI),life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.004540,5,3
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.106220,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.049810,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.845400,6,4
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.433350,18,5
...,...,...,...,...,...,...,...,...,...
186,187,Burundi,0.426,61.6627,10.722722,3.129267,731.786709,4,187
187,188,Central African Republic,0.404,53.8947,8.040172,4.334000,966.058611,1,188
188,189,Niger,0.400,61.5763,6.957112,2.116717,1239.866936,-3,189
189,190,Chad,0.394,52.5254,8.035914,2.573774,1364.169417,-7,190


In [45]:
merge_full = pd.merge(merge, hdi, left_on='country_name_exp', right_on='country_name_hdi', 
                      how='inner')

In [46]:
merge_full

,country_name_exp,country_code,year,BX.GSR.MRCH.CD,country_name_imp,BM.GSR.TOTL.CD,country_name_gdp,NY.GDP.MKTP.CD,country_name_tar,TM.TAX.MRCH.WM.FN.ZS,rank,country_name_hdi,Human Development Index (HDI),life_exp,exp_years_of_school,mean_years_of_school,gni,gni_minus_hdi,old_hdi
0,Afghanistan,AFG,2008,5.632992e+08,Afghanistan,3.873397e+09,Afghanistan,1.024977e+10,Afghanistan,5.00,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
1,Afghanistan,AFG,2012,4.755310e+08,Afghanistan,9.891159e+09,Afghanistan,2.020357e+10,Afghanistan,5.50,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
2,Afghanistan,AFG,2013,5.057926e+08,Afghanistan,1.018433e+10,Afghanistan,2.056449e+10,Afghanistan,5.50,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
3,Afghanistan,AFG,2018,8.752427e+08,Afghanistan,8.086339e+09,Afghanistan,1.841886e+10,Afghanistan,5.72,180,Afghanistan,0.478,61.9824,10.263844,2.985070,1824.190915,-2,177
4,Albania,ALB,1997,9.090000e+07,Albania,6.976125e+08,Albania,2.258514e+09,Albania,14.41,67,Albania,0.796,76.4626,14.448000,11.286455,14131.110390,17,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3011,Zimbabwe,ZWE,2010,3.245066e+09,Zimbabwe,6.593559e+09,Zimbabwe,1.204166e+10,Zimbabwe,16.60,146,Zimbabwe,0.593,59.2531,12.110969,8.710909,3809.887158,9,145
3012,Zimbabwe,ZWE,2011,4.527563e+09,Zimbabwe,9.528102e+09,Zimbabwe,1.410192e+10,Zimbabwe,13.49,146,Zimbabwe,0.593,59.2531,12.110969,8.710909,3809.887158,9,145
3013,Zimbabwe,ZWE,2012,3.963773e+09,Zimbabwe,8.683377e+09,Zimbabwe,1.711485e+10,Zimbabwe,14.76,146,Zimbabwe,0.593,59.2531,12.110969,8.710909,3809.887158,9,145
3014,Zimbabwe,ZWE,2015,3.577478e+09,Zimbabwe,7.584347e+09,Zimbabwe,1.996312e+10,Zimbabwe,12.86,146,Zimbabwe,0.593,59.2531,12.110969,8.710909,3809.887158,9,145


# Database Time

In [ ]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True

In [ ]:
cursor = dbserver.cursor()

In [ ]:
try:
    cursor.execute('CREATE DATABASE world')
except:
    cursor.execute('DROP DATABASE world')
    cursor.execute('CREATE DATABASE world')

In [ ]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'world'
))

In [ ]:
### review if  I even need to merge every 
#worlddf.to_sql('world', con=engine, index=False, chunksize=1000, if_exists = 'replace')

# DBDOCS

In [ ]:
#for col in worlddf.columns:
#    print(col, worlddf[col].dtype)

## gives info for DBdocs

# Dashboard